SET UP môi trường nhé hẹ hẹ

In [2]:
!pip install sentence-transformers faiss-cpu networkx pandas numpy


Load Json luật + Flatten tree


In [2]:
# import json
# from typing import Any, Dict, List, Optional

# def norm(s: Optional[str]) -> str:
#     return (s or "").replace("\r", "").strip()

# def join_path(parts: List[str]) -> str:
#     return " > ".join([p for p in parts if p])

# def flatten_legal_tree(node: Dict[str, Any], path: List[str], out: List[Dict[str, Any]]):
#     ntype = node.get("type", "")
#     title = norm(node.get("title"))
#     content = norm(node.get("content"))

#     new_path = path + ([title] if title else [])
#     if content:
#         out.append({"type": ntype, "path": join_path(new_path), "text": content})

#     for ch in node.get("children", []) or []:
#         flatten_legal_tree(ch, new_path, out)

# def load_legal_json(json_path: str) -> Dict[str, Any]:
#     with open(json_path, "r", encoding="utf-8") as f:
#         return json.load(f)

# def flatten_to_passages(doc: Dict[str, Any]) -> List[str]:
#     info = doc.get("document_info", {})
#     base = [
#         f"Văn bản: {norm(info.get('title'))}",
#         f"URL: {norm(info.get('url'))}",
#         f"Crawled: {norm(info.get('crawled_at'))}",
#     ]

#     units: List[Dict[str, Any]] = []
#     for top in doc.get("body", []) or []:
#         flatten_legal_tree(top, base, units)

#     passages = []
#     for u in units:
#         header = f"[{u['type']}] {u['path']}"
#         passages.append(header + "\n" + u["text"])
#     return passages

# JSON_PATH = "thongtu80.json"  # đổi path
# doc = load_legal_json(JSON_PATH)
# passages = flatten_to_passages(doc)

# print("passages:", len(passages))
# print(passages[0][:800])


Chunking

In [3]:
!pip -q install tiktoken

import tiktoken

In [4]:


# class TokenSplitter:
#     def __init__(self, chunk_tokens=800, overlap_tokens=80, tokenizer_name="cl100k_base"):
#         self.enc = tiktoken.get_encoding(tokenizer_name)
#         self.chunk_tokens = chunk_tokens
#         self.overlap_tokens = overlap_tokens

#     def chunk(self, text: str):
#         toks = self.enc.encode(text)
#         step = max(1, self.chunk_tokens - self.overlap_tokens)
#         out = []
#         for i in range(0, len(toks), step):
#             sub = toks[i:i+self.chunk_tokens]
#             if sub:
#                 out.append(self.enc.decode(sub))
#         return out

# splitter = TokenSplitter(chunk_tokens=800, overlap_tokens=80)

# dataset = []
# for p in passages:
#     dataset.extend(splitter.chunk(p))

# print("chunks:", len(dataset))
# print(dataset[0][:600])

Embedding tiếng Việt (E5 multilingual) + FAISS index

In [5]:
# import numpy as np
# import faiss
# from sentence_transformers import SentenceTransformer

# embed_model_name = "intfloat/multilingual-e5-base"  # hoặc multilingual-e5-large
# embedder = SentenceTransformer(embed_model_name)

# def embed_texts(texts, batch_size=32):
#     # E5: prefix passage:
#     inputs = [f"passage: {t}" for t in texts]
#     vecs = embedder.encode(inputs, batch_size=batch_size, normalize_embeddings=True)
#     return np.asarray(vecs, dtype="float32")

# embeddings = embed_texts(dataset, batch_size=32)
# print("embeddings:", embeddings.shape)

# d = embeddings.shape[1]
# index = faiss.IndexFlatIP(d)
# index.add(embeddings)
# print("FAISS added:", index.ntotal)

NANOVectorDB

In [6]:
# !pip -q install nano-vectordb

# from nano_vectordb import NanoVectorDB

# vdb = NanoVectorDB(d, storage_file="vdb_chunks.json")

# # upsert
# datas = []
# for i, (text, vec) in enumerate(zip(dataset, embeddings)):
#     datas.append({"__id__": f"chunk-{i}", "__vector__": vec, "content": text})

# vdb.upsert(datas=datas)
# vdb.save()
# print("saved vdb_chunks.json")


LLM extractor (JSON schema)

In [7]:
# import re
# import networkx as nx

# def llm_extract_entities_relations(chunk: str) -> Dict[str, Any]:
#     prompt = f"""
# Bạn là hệ thống trích xuất tri thức cho văn bản luật Việt Nam.

# Từ đoạn văn sau, hãy trích xuất:
# - entities: danh sách thực thể quan trọng (cơ quan, đối tượng, khái niệm, văn bản pháp luật, thủ tục, hành vi, nghĩa vụ...)
# - relations: danh sách quan hệ dạng (head, relation, tail)

# QUY TẮC:
# - Ưu tiên các khái niệm pháp lý: "người nộp thuế", "cơ quan thuế", "hồ sơ hoàn thuế", "thương mại điện tử", "nền tảng số"...
# - Hạn chế tạo entity kiểu "Điều 7", "khoản 1" (chỉ giữ nếu liên quan trực tiếp).
# - relation nên là động từ/cụm ngắn: "được hiểu là", "bao gồm", "quy định", "hướng dẫn", "quản lý", "áp dụng cho"...
# - Output phải là JSON thuần theo format:

# {{
#   "entities": ["..."],
#   "relations": [
#     {{"head":"...", "relation":"...", "tail":"..."}}
#   ]
# }}

# ĐOẠN VĂN:
# \"\"\"{chunk}\"\"\"
# JSON:
# """
#     text = llm_generate(prompt, max_tokens=900)
#     # cố parse JSON robust
#     try:
#         obj = json.loads(text)
#         if "entities" not in obj: obj["entities"] = []
#         if "relations" not in obj: obj["relations"] = []
#         return obj
#     except Exception:
#         # fallback: cố lấy json block
#         m = re.search(r"\{.*\}", text, flags=re.S)
#         if m:
#             try:
#                 return json.loads(m.group(0))
#             except Exception:
#                 pass
#         return {"entities": [], "relations": []}

# # build KG
# kg = nx.DiGraph()
# chunk_entities = []

# for i, ch in enumerate(dataset[:200]):  # test 200 chunk trước cho nhanh
#     extracted = llm_extract_entities_relations(ch)
#     ents = set([e.strip() for e in extracted.get("entities", []) if e and isinstance(e, str)])
#     chunk_entities.append(ents)

#     for e in ents:
#         if not kg.has_node(e):
#             kg.add_node(e)

#     for r in extracted.get("relations", []) or []:
#         h = (r.get("head") or "").strip()
#         rel = (r.get("relation") or "").strip()
#         t = (r.get("tail") or "").strip()
#         if h and t:
#             kg.add_edge(h, t, relation=rel or "related_to", source_chunk=i)

# print("KG nodes:", kg.number_of_nodes())
# print("KG edges:", kg.number_of_edges())


In [4]:
import os

# set key ở đây (đừng hardcode trong code dài)
os.environ["NVAPI_KEY"] = "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"  # <-- điền key của bạn

JSON_PATH = "thongtu80.json"
CACHE_DIR = "./tt80_cache_full"   # thư mục lưu mọi thứ


In [5]:
!pip install openai

In [6]:
import os, json, re, logging
from datetime import datetime
from typing import Any, Dict, List, Optional, Set, Tuple

import numpy as np
import networkx as nx
import faiss
from sentence_transformers import SentenceTransformer

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("OFFLINE_BUILD_NO_LLM")

# ====== CONFIG ======
JSON_PATH  = "thongtu80.json"
CACHE_DIR  = "./tt80_artifacts_no_llm"

EMBED_MODEL = "intfloat/multilingual-e5-base"
BATCH_EMBED = 32

CHUNK_TOKENS = 800
CHUNK_OVERLAP = 80
CHUNK_CHAR = 1200
CHUNK_CHAR_OVERLAP = 120

# KG params (offline)
TOP_K_TERMS_PER_CHUNK = 18
MIN_TERM_LEN = 3
MAX_TERM_WORDS = 6
COOC_WINDOW = 2  # connect term i with i+1..i+COOC_WINDOW in the same chunk

# ==============
# Utils: Flatten legal JSON
# ==============
def norm(s: Optional[str]) -> str:
    return (s or "").replace("\r", "").strip()

def join_path(parts: List[str]) -> str:
    return " > ".join([p for p in parts if p])

def flatten_legal_tree(node: Dict[str, Any], path: List[str], out: List[Dict[str, Any]]):
    ntype = node.get("type", "")
    title = norm(node.get("title"))
    content = norm(node.get("content"))
    new_path = path + ([title] if title else [])
    if content:
        out.append({"type": ntype, "path": join_path(new_path), "text": content})
    for ch in node.get("children", []) or []:
        flatten_legal_tree(ch, new_path, out)

def load_legal_json(json_path: str) -> Dict[str, Any]:
    with open(json_path, "r", encoding="utf-8") as f:
        return json.load(f)

def flatten_to_passages(doc: Dict[str, Any]) -> List[str]:
    info = doc.get("document_info", {})
    base = [
        f"Văn bản: {norm(info.get('title'))}",
        f"URL: {norm(info.get('url'))}",
        f"Crawled: {norm(info.get('crawled_at'))}",
    ]
    units: List[Dict[str, Any]] = []
    for top in doc.get("body", []) or []:
        flatten_legal_tree(top, base, units)

    passages = []
    for u in units:
        header = f"[{u['type']}] {u['path']}"
        passages.append(header + "\n" + u["text"])
    return passages

# ==============
# Chunking (token if tiktoken exists else char)
# ==============
def try_import_tiktoken():
    try:
        import tiktoken
        return tiktoken
    except Exception:
        return None

def token_chunk(text: str, chunk_tokens: int, overlap: int, tokenizer_name="cl100k_base") -> List[str]:
    tiktoken = try_import_tiktoken()
    if tiktoken is None:
        raise ModuleNotFoundError("tiktoken not installed")
    enc = tiktoken.get_encoding(tokenizer_name)
    toks = enc.encode(text)
    step = max(1, chunk_tokens - overlap)
    out = []
    for i in range(0, len(toks), step):
        sub = toks[i:i+chunk_tokens]
        if sub:
            out.append(enc.decode(sub))
    return out

def char_chunk(text: str, chunk_size: int, overlap: int) -> List[str]:
    seps = ["\n\n", "\n", ". ", "; ", ", ", " ", ""]
    chunks: List[str] = []
    buf = text.strip()
    if not buf:
        return []
    start = 0
    while start < len(buf):
        end = min(len(buf), start + chunk_size)
        piece = buf[start:end]
        cut = -1
        for sep in seps:
            idx = piece.rfind(sep)
            if idx > cut:
                cut = idx
        if cut <= 0:
            cut = len(piece)
        else:
            cut = cut + 1
        chunk = piece[:cut].strip()
        if chunk:
            chunks.append(chunk)
        start = start + max(1, cut - overlap)
    return chunks

def make_chunks(passages: List[str]) -> List[str]:
    dataset: List[str] = []
    use_token = try_import_tiktoken() is not None
    logger.info(f"Chunking mode: {'tiktoken' if use_token else 'char-fallback'}")
    for p in passages:
        if use_token:
            try:
                dataset.extend(token_chunk(p, CHUNK_TOKENS, CHUNK_OVERLAP))
            except Exception:
                dataset.extend(char_chunk(p, CHUNK_CHAR, CHUNK_CHAR_OVERLAP))
        else:
            dataset.extend(char_chunk(p, CHUNK_CHAR, CHUNK_CHAR_OVERLAP))
    return dataset

# ==============
# Embedding + FAISS
# ==============
embedder = SentenceTransformer(EMBED_MODEL)

def embed_texts(texts: List[str], batch_size=32) -> np.ndarray:
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inp = [f"passage: {t}" for t in batch]
        v = embedder.encode(inp, normalize_embeddings=True)
        vecs.append(np.asarray(v, dtype="float32"))
    return np.vstack(vecs)

def build_faiss(embeddings: np.ndarray) -> faiss.Index:
    d = embeddings.shape[1]
    idx = faiss.IndexFlatIP(d)
    idx.add(embeddings.astype("float32"))
    return idx

# ==============
# OFFLINE Entity / Term extraction (NO LLM)
# ==============
def normalize_term(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    # remove surrounding quotes
    s = s.strip("“”\"' ")
    return s

def extract_quoted_terms(text: str) -> List[str]:
    # captures “...”, "...", '...'
    terms = []
    for a,b,c in re.findall(r"“([^”]{2,120})”|\"([^\"]{2,120})\"|'([^']{2,120})'", text):
        t = normalize_term(a or b or c)
        if t:
            terms.append(t)
    return terms

def extract_legal_markers(text: str) -> List[str]:
    # keep lawy concepts + headings
    # examples: Điều 1, Khoản 2, điểm a, Nghị định số 126/2020/NĐ-CP
    pats = [
        r"(Điều\s+\d+[A-Za-z]?)",
        r"(Khoản\s+\d+[A-Za-z]?)",
        r"(điểm\s+[a-zđ])",
        r"(Nghị\s+định\s+số\s+\d+\/\d+\/[A-ZĐ\-]+)",
        r"(Thông\s+tư\s+\d+\/\d+\/[A-ZĐ\-]+)",
        r"(Luật\s+[A-ZÀ-Ỹa-zà-ỹ\s]{3,80})",
    ]
    out = []
    for pat in pats:
        out.extend([normalize_term(x) for x in re.findall(pat, text, flags=re.I)])
    return out

def extract_terms_yake(text: str, top_k: int) -> List[str]:
    try:
        import yake
        kw_extractor = yake.KeywordExtractor(lan="vi", n=MAX_TERM_WORDS, top=top_k)
        kws = kw_extractor.extract_keywords(text)
        # yake returns list[(kw, score)] lower score better
        kws = sorted(kws, key=lambda x: x[1])[:top_k]
        return [normalize_term(k) for k,_ in kws if k]
    except Exception:
        return []

def extract_terms_underthesea(text: str, top_k: int) -> List[str]:
    # optional: better Vietnamese word segmentation + noun phrases
    try:
        from underthesea import pos_tag
        tags = pos_tag(text)
        # collect sequences of N / NP / A (rough)
        terms = []
        buf = []
        for w,t in tags:
            if t in ("N", "Np", "Ny", "Nc", "A"):
                buf.append(w)
            else:
                if 1 <= len(buf) <= MAX_TERM_WORDS:
                    terms.append(normalize_term(" ".join(buf)))
                buf = []
        if 1 <= len(buf) <= MAX_TERM_WORDS:
            terms.append(normalize_term(" ".join(buf)))
        # filter length
        terms = [x for x in terms if len(x) >= MIN_TERM_LEN]
        # unique preserve order
        seen = set()
        out = []
        for x in terms:
            lx = x.lower()
            if lx not in seen:
                seen.add(lx)
                out.append(x)
        return out[:top_k]
    except Exception:
        return []

def extract_terms_offline(text: str, top_k: int = TOP_K_TERMS_PER_CHUNK) -> List[str]:
    # Priority order: quoted terms + legal markers + underthesea + yake + fallback regex ngrams
    terms = []
    terms.extend(extract_quoted_terms(text))
    terms.extend(extract_legal_markers(text))

    # underthesea noun-ish phrases
    ut = extract_terms_underthesea(text, top_k=top_k)
    if ut:
        terms.extend(ut)

    # yake keyphrases
    yk = extract_terms_yake(text, top_k=top_k)
    if yk:
        terms.extend(yk)

    # fallback: common patterns like "cơ quan thuế", "người nộp thuế" by scanning
    # pick 2-5 word grams around these trigger words
    triggers = ["thuế", "hồ sơ", "nghĩa vụ", "quy định", "cơ quan", "người", "thương mại", "nền tảng", "hoàn"]
    words = re.findall(r"[A-Za-zÀ-ỹ0-9_]+", text.lower())
    for i,w in enumerate(words):
        if w in triggers:
            for n in (2,3,4):
                gram = " ".join(words[i:i+n])
                gram = normalize_term(gram)
                if len(gram) >= MIN_TERM_LEN:
                    terms.append(gram)

    # clean + unique
    clean = []
    seen = set()
    for t in terms:
        t = normalize_term(t)
        if not t:
            continue
        if len(t) < MIN_TERM_LEN:
            continue
        if len(t.split()) > MAX_TERM_WORDS:
            continue
        key = t.lower()
        if key not in seen:
            seen.add(key)
            clean.append(t)

    return clean[:top_k]

# ==============
# Build KG (NO LLM): co-occurrence graph
# ==============
def build_kg_and_chunk_entities_no_llm(dataset: List[str]) -> Tuple[nx.DiGraph, List[Set[str]]]:
    kg = nx.DiGraph()
    chunk_entities: List[Set[str]] = []

    for i, chunk in enumerate(dataset):
        terms = extract_terms_offline(chunk, top_k=TOP_K_TERMS_PER_CHUNK)
        ent_set = set(terms)
        chunk_entities.append(ent_set)

        for t in ent_set:
            if not kg.has_node(t):
                kg.add_node(t)

        # co-occurrence edges (ordered terms)
        ordered = terms
        for a_idx, a in enumerate(ordered):
            for b_idx in range(a_idx+1, min(len(ordered), a_idx+1+COOC_WINDOW)):
                b = ordered[b_idx]
                if a == b:
                    continue
                # directional edge with relation label
                if kg.has_edge(a, b):
                    kg[a][b]["weight"] = kg[a][b].get("weight", 1) + 1
                else:
                    kg.add_edge(a, b, relation="co_occurs_with", weight=1, source_chunk=i)

        if (i+1) % 200 == 0:
            logger.info(f"KG progress {i+1}/{len(dataset)} | nodes={kg.number_of_nodes()} edges={kg.number_of_edges()}")

    return kg, chunk_entities

# ====== RUN OFFLINE BUILD ======
os.makedirs(CACHE_DIR, exist_ok=True)

logger.info("Load JSON -> passages")
doc = load_legal_json(JSON_PATH)
passages = flatten_to_passages(doc)

logger.info("Chunking")
dataset = make_chunks(passages)
logger.info(f"chunks = {len(dataset)}")

logger.info("Embedding + FAISS")
embeddings = embed_texts(dataset, batch_size=BATCH_EMBED)
index = build_faiss(embeddings)
logger.info(f"FAISS ntotal = {index.ntotal}")

logger.info("Build KG + chunk_entities (NO LLM)")
kg, chunk_entities = build_kg_and_chunk_entities_no_llm(dataset)

# ====== SAVE ARTIFACTS ======
with open(os.path.join(CACHE_DIR, "chunks.json"), "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False)

faiss.write_index(index, os.path.join(CACHE_DIR, "faiss.index"))
np.save(os.path.join(CACHE_DIR, "embeddings.npy"), embeddings)  # optional

# nx.write_gpickle(kg, os.path.join(CACHE_DIR, "kg.pkl"))

import pickle

with open(os.path.join(CACHE_DIR, "kg.pkl"), "wb") as f:
    pickle.dump(kg, f)

with open(os.path.join(CACHE_DIR, "chunk_entities.json"), "w", encoding="utf-8") as f:
    json.dump([list(s) for s in chunk_entities], f, ensure_ascii=False)

meta = {
    "created_at": datetime.now().isoformat(),
    "embedding_model": EMBED_MODEL,
    "num_chunks": len(dataset),
    "kg_nodes": kg.number_of_nodes(),
    "kg_edges": kg.number_of_edges(),
    "build_mode": "NO_LLM",
}
with open(os.path.join(CACHE_DIR, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

logger.info(f"✅ DONE. Saved artifacts to: {CACHE_DIR}")
logger.info(f"KG nodes={kg.number_of_nodes()} edges={kg.number_of_edges()}")


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-base
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/intfloat/multilingual-e5-base/835193815a3936a24a0ee7dc9e3d48c1fbb19c55/modules.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/config_sentence_transformers.json "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/intfloat/multilingual-e5-base/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https:/

In [9]:
import os, json, re
import numpy as np
import networkx as nx
import faiss
from sentence_transformers import SentenceTransformer
from openai import OpenAI

CACHE_DIR = "./tt80_artifacts_no_llm"
MODEL_ANSWER = "moonshotai/kimi-k2-instruct-0905"
TOP_K = 6
ALPHA = 0.85

# --- load artifacts once ---
if "BOT_READY" not in globals():
    with open(os.path.join(CACHE_DIR, "meta.json"), "r", encoding="utf-8") as f:
        META = json.load(f)

    with open(os.path.join(CACHE_DIR, "chunks.json"), "r", encoding="utf-8") as f:
        DATASET = json.load(f)

    INDEX = faiss.read_index(os.path.join(CACHE_DIR, "faiss.index"))
    # KG = nx.read_gpickle(os.path.join(CACHE_DIR, "kg.pkl"))
    
    import pickle

    with open(os.path.join(CACHE_DIR, "kg.pkl"), "rb") as f:
        KG = pickle.load(f)

    

    with open(os.path.join(CACHE_DIR, "chunk_entities.json"), "r", encoding="utf-8") as f:
        CHUNK_ENTITIES = [set(x) for x in json.load(f)]

    EMBEDDER = SentenceTransformer(META["embedding_model"])
    BOT_READY = True

    print("✅ Artifacts loaded")
    print("chunks:", len(DATASET), "| KG nodes:", KG.number_of_nodes(), "| KG edges:", KG.number_of_edges())

# --- LLM client (ONLY for answering) ---
if not os.getenv("NVAPI_KEY"):
    raise RuntimeError("Missing NVAPI_KEY env var.")
client = OpenAI(base_url="https://integrate.api.nvidia.com/v1", api_key=os.getenv("NVAPI_KEY"))

def llm_answer(prompt: str, temperature=0.2, max_tokens=450) -> str:
    resp = client.chat.completions.create(
        model=MODEL_ANSWER,
        messages=[{"role":"user","content":prompt}],
        temperature=temperature,
        top_p=0.9,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content.strip()

# --- fast keyword/entity extraction without LLM ---
STOP = set(["là", "và", "của", "theo", "đối", "với", "như", "thế", "nào", "gì", "bao", "gồm", "quy", "định", "hướng", "dẫn", "không", "có", "một", "các", "được"])

def fast_terms(query: str) -> Set[str]:
    q = query.lower()
    # keep quoted phrases
    quoted = re.findall(r"“([^”]+)”|\"([^\"]+)\"", query)
    terms = set()
    for a,b in quoted:
        s = (a or b).strip().lower()
        if s:
            terms.add(s)
    # split words
    words = re.findall(r"[a-zA-ZÀ-ỹ0-9_]+", q)
    for w in words:
        if len(w) >= 3 and w not in STOP:
            terms.add(w)
    return terms

def semantic_search(query: str, top_k: int):
    q_vec = EMBEDDER.encode([f"query: {query}"], normalize_embeddings=True).astype("float32")
    D, I = INDEX.search(q_vec, top_k)
    return [(int(I[0][j]), float(D[0][j])) for j in range(top_k)]

def graph_scores(query_terms: Set[str]) -> np.ndarray:
    scores = np.zeros(len(DATASET), dtype=float)
    if not query_terms:
        return scores
    for i, ents in enumerate(CHUNK_ENTITIES):
        # match by substring (legal terms)
        hit = 0
        for t in query_terms:
            for e in ents:
                if t in e.lower():
                    hit += 1
                    break
        scores[i] = hit
    if scores.max() > 0:
        scores = scores / (scores.max() + 1e-12)
    return scores

def kg_facts_from_terms(query_terms: Set[str], limit=10) -> List[str]:
    facts = []
    # fuzzy: find nodes containing any term
    for t in list(query_terms)[:8]:
        cand = [n for n in KG.nodes() if t in str(n).lower()]
        for node in cand[:2]:
            for nb in list(KG.successors(node))[:3]:
                rel = KG.get_edge_data(node, nb).get("relation", "related_to")
                facts.append(f"{node} --{rel}--> {nb}")
                if len(facts) >= limit:
                    return facts
    return facts

def retrieve_context(query: str, top_k=TOP_K, alpha=ALPHA):
    sem = semantic_search(query, top_k=top_k*2)
    terms = fast_terms(query)
    gs = graph_scores(terms)

    combined = []
    for idx, sem_score in sem:
        g = float(gs[idx])
        combined.append((idx, alpha*sem_score + (1-alpha)*g))
    combined.sort(key=lambda x: x[1], reverse=True)
    idxs = [i for i,_ in combined[:top_k]]

    ctx = "\n\n".join([DATASET[i] for i in idxs])
    facts = kg_facts_from_terms(terms, limit=10)
    if facts:
        ctx += "\n\nKG_FACTS:\n" + "\n".join(facts)
    return ctx, idxs

def ask(query: str):
    ctx, idxs = retrieve_context(query)
    prompt = f"""
Bạn là trợ lý hỏi đáp luật Việt Nam.

sử dụng những kiến thức từ knowledge graph, không tự bịa đặt nhé

hơn nữa, những kiến thức có trong CONTEXT rất quan trọng, phải dùng đến chúng để trả lời câu hỏi. Nên hãy dùng hết để tóm tắt để trả lời

CHỈ trả lời dựa trên CONTEXT. Không bịa.
Nếu CONTEXT không có thông tin, trả lời đúng 1 dòng:
no relevant information found

CONTEXT:
{ctx}

QUESTION:
{query}

ANSWER:
"""
    ans = llm_answer(prompt, temperature=0.2, max_tokens=450)
    return ans, idxs

print("\n✅ Chatbot mode ready. Type question, 'exit' to stop.\n")
# while True:
#     q = input("You: ").strip()
#     if not q:
#         continue
#     if q.lower() in ("exit","quit","q"):
#         print("Bye!")
#         break
#     ans, idxs = ask(q)
#     print(f"\nBot: {ans}\n")






✅ Chatbot mode ready. Type question, 'exit' to stop.



In [10]:
question = "Hoạt động thương mại điện tử được hiểu như thế nào?"

context, used_chunks = retrieve_context(question)

print("QUESTION:")
print(question)

print("\nCONTEXT USED:")
print(context)

print("\nANSWER:")
print(llm_answer(f"""
Bạn là trợ lý hỏi đáp luật Việt Nam.

CHỈ trả lời dựa trên CONTEXT.
Nếu không có thông tin, trả lời:
no relevant information found

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""))

print("\nUSED CHUNKS:")
print(used_chunks)


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.26it/s]


QUESTION:
Hoạt động thương mại điện tử được hiểu như thế nào?

CONTEXT USED:
[clause] Văn bản: Thông tư 80/2021/TT-BTC hướng dẫn Luật Quản lý thuế  Nghị định 126/2020/NĐ-CP > URL: https://thuvienphapluat.vn/van-ban/Thue-Phi-Le-Phi/Thong-tu-80-2021-TT-BTC-huong-dan-Luat-Quan-ly-thue-Nghi-dinh-126-2020-ND-CP-466716.aspx > Crawled: 2026-01-24 > Chương I
NHỮNG QUY ĐỊNH CHUNG > Điều 2. Đối tượng áp dụng Đối tượng áp dụng Thông tư này bao gồm: Người nộp thuế; cơ quan thuế; công chức thuế; cơ quan nhà nước, tổ chức, cá nhân khác có liên quan theo quy định tại Điều 2 Luật Quản lý thuế. Điều 3. Giải thích từ ngữ > 1
“Hoạt động
thương mại điện tử” là việc tiến hành một phần hoặc toàn bộ quy trình của hoạt
động thương mại bằng phương tiện điện tử có kết nối với mạng Internet, mạng viễn
thông di động hoặc các mạng mở khác theo quy định tại Nghị định số 52/2013/NĐ-CP ngày 16/5/2013 của Chính phủ.

[clause] Văn bản: Thông tư 80/2021/TT-BTC hướng dẫn Luật Quản lý thuế  Nghị định 126/2020/NĐ-CP > URL:

INFO:httpx:HTTP Request: POST https://integrate.api.nvidia.com/v1/chat/completions "HTTP/1.1 200 OK"


“Hoạt động thương mại điện tử” là việc tiến hành một phần hoặc toàn bộ quy trình của hoạt động thương mại bằng phương tiện điện tử có kết nối với mạng Internet, mạng viễn thông di động hoặc các mạng mở khác theo quy định tại Nghị định số 52/2013/NĐ-CP ngày 16/5/2013 của Chính phủ.

USED CHUNKS:
[3, 1166, 1205, 1206, 1165, 1167]
